# batch makes all queries to get top 40 products relevant for each query

In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from joblib import Parallel, delayed
import polars as pl


In [2]:
import sys
sys.executable

'/opt/conda/envs/p312/bin/python'

In [3]:
model = 'openai'
locale = 'us'

In [6]:
q = pd.read_parquet(f'{model}/queries-{locale}.parquet')

In [4]:
p = pl.read_parquet(f'{model}/products-{locale}.parquet')
p = p.to_pandas()


In [7]:
q.shape, p.shape

((97345, 2), (1215851, 3))

In [8]:
qe = np.stack(q.embeddings.values.copy())
pe = np.stack(p.embeddings.values.copy())
qe.shape, pe.shape

((97345, 3072), (1215851, 3072))

In [9]:
def get_dotp_closests_idxs(q_embedding,p_embeddings_matrix, topk=40):
    return np.argsort( p_embeddings_matrix.dot(q_embedding) )[-topk:][::-1]


In [10]:
1

1

In [11]:
qnn = Parallel(n_jobs=-1, verbose=5)(delayed(get_dotp_closests_idxs)(qi,pe) for qi in qe)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 64 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   51.8s
[Parallel(n_jobs=-1)]: Done 160 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done 322 tasks      | elapsed:  2.2min
[Parallel(n_jobs=-1)]: Done 520 tasks      | elapsed:  3.1min
[Parallel(n_jobs=-1)]: Done 754 tasks      | elapsed:  4.1min
[Parallel(n_jobs=-1)]: Done 1024 tasks      | elapsed:  5.3min
[Parallel(n_jobs=-1)]: Done 1330 tasks      | elapsed:  6.7min
[Parallel(n_jobs=-1)]: Done 1672 tasks      | elapsed:  8.2min
[Parallel(n_jobs=-1)]: Done 2050 tasks      | elapsed:  9.9min
[Parallel(n_jobs=-1)]: Done 2464 tasks      | elapsed: 11.7min
[Parallel(n_jobs=-1)]: Done 2914 tasks      | elapsed: 13.7min
[Parallel(n_jobs=-1)]: Done 3400 tasks      | elapsed: 15.9min
[Parallel(n_jobs=-1)]: Done 3922 tasks      | elapsed: 18.3min
[Parallel(n_jobs=-1)]: Done 4480 tasks      | elapsed: 20.8min
[Parallel(n_jobs=-1)]: Done 5074 tasks      | 

In [17]:
p.index = p.__index_level_0__

In [19]:
q['top40_products'] = [list(p.index[qnni]) for qnni in tqdm(qnn)]

100%|█████████████████████████████████████████████████████████████████████████████████████████████| 97345/97345 [00:00<00:00, 102032.28it/s]


In [20]:
qr = []
for query_id, qi in tqdm(q.iterrows()):
    for product_id in qi.top40_products:
        qr.append([query_id, product_id])
    

97345it [00:06, 13927.65it/s]


In [21]:
pd.DataFrame(qr, columns = ['query_id', 'product_id']).to_csv(f'{model}/embeddings_dotp_ranking_{locale}.csv', index=False)

In [18]:
p

,product,embeddings,__index_level_0__
__index_level_0__,,,
0000306045,<TITLE>WhiteCoat Clipboard- Pink - Respiratory...,"[-0.027882717549800873, -0.021610360592603683,...",0000306045
0002159511,<TITLE>A Day in the Life of Ireland</TITLE>\n<...,"[0.03405754268169403, 0.005424880422651768, -0...",0002159511
0003002764,<TITLE>HENRY'S AWFUL MISTAKE</TITLE>,"[0.022827627137303352, -0.014154857955873013, ...",0003002764
0004126475,<TITLE>David Bellamy's Watercolour Landscape C...,"[0.049933984875679016, 0.028047902509570122, -...",0004126475
0004127579,<TITLE>Developing Your Watercolours</TITLE>\n<...,"[0.030019331723451614, 0.031795453280210495, -...",0004127579
...,...,...,...
BT00CTOZCE,<TITLE>Amazon.com $100 Gift Card in a Greeting...,"[-0.014043739065527916, -0.0016380960587412119...",BT00CTOZCE
BT00CTP2VW,<TITLE>Amazon.com $100 Gift Card in a Greeting...,"[-0.013048318214714527, -0.017103096470236778,...",BT00CTP2VW
BT00KQVWMK,<TITLE>Clif Bar Energy White Chocolate Macadam...,"[-0.0005861932877451181, -0.02896728552877903,...",BT00KQVWMK
